In [3]:
from Levenshtein import ratio, jaro, distance
from collections import Counter

import pandas as pd
import requests
import subprocess
import gzip
import json
import math
import os
import re

def set_default_path(directory: str | None = 'Event-Aggregator') -> str:
    """Changes the working directory to """
    while not os.getcwd().endswith(directory):
        os.chdir('..')
        if len(os.getcwd()) == 0:
            raise FileExistsError
    return os.getcwd()

def query(url, data, mode):
    match mode:
        case 'POST':
            item = requests.post(url,json=data)
            item = json.loads(item.content.decode())
        case 'GET':
            item = requests.get(url)
            item = json.loads(item.content.decode())
    return item

default_path = os.path.join(set_default_path() ,'Production_Environment')
server_path  = os.path.join(default_path,'Servers') 
collectors_path = os.path.join(server_path,'Collectors')
events_path  = os.path.join(collectors_path,'Events')
data_path    = os.path.join(events_path,'Data')
File_path    = os.path.join(data_path, 'Final_Events.csv')

In [5]:
df = pd.read_csv(os.path.join(data_path,'Final_Events.csv'))

In [6]:
n_rows, n_columns = df.shape
n_rows,n_columns

(489, 9)

In [76]:
results  = []
for idx in range(n_rows):
    row = df.iloc[idx].fillna('')
    item = {
        'TITLE': row['Title'],
        'DATE' : row['Start Date'],
        'MAINPAGE' : row['Main Page'],
        'ADDRESS1' : row['Address1'],
        'ADDRESS2' : row['Address2'],
        'WEHN' : row['When'],
        'THUMB': row['thumb'],
        'IMAGE' : row['image'],
        'DESC' : row['Desc'],
        'LongDesc': ''
    }
    results.append(item)


In [17]:
url  = "http://127.0.0.1:5000/get_descs"
    
data = query(url = url, data = None, mode = 'GET')
IDS, desc = zip(*data)


In [18]:
import numpy as np

In [21]:
np.array(desc)

array(['Full information about PGL Wallachia S3 Dota 2. Match results, VODs, streams, team rosters, schedules',
       'Full information about BLAST Slam #3 Dota 2. Match results, VODs, streams, team rosters, schedules',
       'Full information about ESL One Raleigh 2025 Dota 2. Match results, VODs, streams, team rosters, schedules',
       'Full information about PGL Wallachia S4 Dota 2. Match results, VODs, streams, team rosters, schedules',
       'Find tickets for Dota at Weingut Karl Sonntag in Nittel on 7/27/2025 at 7:00 PM',
       'Find tickets for Dota with The Q4 at Karlstorbahnhof Heidelberg in Heidelberg on 11/22/2025 at 8:00 PM',
       '🎶 The Ultimate Dota Kehr Experience LIVE at Tollhaus—This Friday, November 21, 2025! 🎶 It’s that time of the year where everyone wants to have a musical breath! Your favorite, Dota Kehr is...',
       '',
       'Find tickets for Dota at E-Werk Saal in Erlangen on 11/18/2025 at 8:00 PM',
       'Join us every Monday from 6p - 8p for Espor

In [12]:
url  = "http://127.0.0.1:5000/get_col"
    
select = {
    'COL': 'DESC'
}
descriptions  = query(url = url, data = select, mode = 'POST')